<a href="https://colab.research.google.com/github/JorgeSedek/tp2_orga_datos/blob/main/TP2_S39.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 52 kB/s 
     |████████████████████████████████| 199 kB 58.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2b3cdcae097c3da0087355d00271ca0c6901f5a9f7579de9382f5bcbcf98bb22
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
rdd_movies = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/movies.parquet').rdd
rdd_compania = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/companies.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


S39 () Obtenga un vector que indique si una compañía trabajo/no trabajo en una película, para cada película. Para cada compañía utilice estos vectores para buscar cuál es la compañía más similar, utilizando la distancia coseno. Imprima toda la lista de tuplas (compañía, similar).

In [ ]:
rdd_pelis_companias = rdd_movies.filter(lambda x: not pd.isnull(x.id) and (not pd.isnull(x.production_companies)))\
.map(lambda x: ((x.id.split(",")), (x.production_companies.split(","))))

In [ ]:
rdd_companias_pelis = rdd_pelis_companias.flatMap(lambda x:  (((x[1][i], x[0])) for i in range(len(x[1]))))

In [ ]:
rdd_companias_pelis2 = rdd_companias_pelis.map(lambda x: (x[0], (x[1])))

In [ ]:
rdd_companias_pelis3 = rdd_companias_pelis2.join(rdd_compania.map(lambda x: (str(x[0]),x[1])))

se obtiene (compania, lista de peliculas en que trabajo la compania)

In [ ]:
rdd_companias_pelis3.first()

('Paramount Pictures',
 ['11860',
  '9603',
  '9089',
  '11359',
  '17182',
  '13997',
  '2086',
  '12158',
  '11066',
  '11863',
  '10329',
  '49805',
  '4954',
  '11777',
  '28313',
  '8986',
  '11593',
  '28033',
  '193',
  '11381',
  '1587',
  '9271',
  '9331',
  '13',
  '36593',
  '2758',
  '306',
  '19819',
  '9612',
  '18551',
  '37233',
  '31643',
  '29918',
  '867',
  '21352',
  '9066',
  '251',
  '1592',
  '954',
  '18414',
  '8069',
  '10280',
  '9826',
  '32872',
  '38223',
  '12606',
  '10061',
  '238',
  '11015',
  '27854',
  '13320',
  '164',
  '426',
  '567',
  '6620',
  '804',
  '599',
  '121357',
  '381',
  '22649',
  '252',
  '2119',
  '744',
  '127973',
  '11455',
  '85',
  '335',
  '539',
  '240',
  '343',
  '829',
  '3063',
  '15143',
  '89',
  '24913',
  '8851',
  '199',
  '152',
  '174',
  '172',
  '154',
  '157',
  '168',
  '621',
  '9037',
  '46027',
  '3179',
  '30285',
  '27265',
  '17894',
  '9403',
  '10003',
  '2163',
  '754',
  '8413',
  '1669',
  '10806

In [ ]:
rdd_companias_pelis4 = rdd_companias_pelis3.map(lambda x: (x[1][1], x[1][0]))

In [ ]:
rdd_companias_pelis3 = rdd_companias_pelis4.reduceByKey(lambda a,b: a + b)

In [ ]:
rdd_companias_pelis4 = rdd_companias_pelis3.map(lambda x: (x[0],set(x[1])))

In [ ]:
rdd_companias_pelis4.take(2)

In [ ]:
rdd_companias_pelis5 = rdd_companias_pelis4.cartesian(rdd_companias_pelis4).filter(lambda x: not (x[0][0] == x[1][0]))

In [ ]:
import math

def distancia_coseno(set1,set2):
  return (len(set1.intersection(set2))/((math.sqrt(len(set1)))*(math.sqrt(len(set2)))))

set1 = {"peli1","peli2","peli3"}
set2 = {"peli2","peli4"}

distancia_coseno(set1,set2)


0.40824829046386296

In [ ]:
rdd_companias_pelis6 = rdd_companias_pelis5.map(lambda x: (x[0][0],(x[1][0],distancia_coseno(x[0][1],x[1][1])))).filter(lambda x: not (x[1][1] == 0))\
.reduceByKey(lambda a,b: a if a[1] > b[1] else b)

In [ ]:
rdd_companias_pelis6.collect()

[('Metro-Goldwyn-Mayer (MGM)', ("Loew's", 0.155591021938952)),
 ('PolyGram Filmed Entertainment', ('Propaganda Films', 0.2667891875399663)),
 ('Allegro Films', ('Westwind', 0.2773500981126146)),
 ('Kennedy Miller Productions', ('Mad Max Films', 0.30151134457776363)),
 ('Filmwerks', ('Imperial Entertainment', 0.5345224838248487)),
 ('RCS Films & TV', ('N.E.F. Filmproduktion und Vertriebs GmbH', 1.0)),
 ('west wind entertainment', ('Echelon Studios', 0.7071067811865475)),
 ('New Zealand On Air',
  ('Village Roadshow Pictures Worldwide', 0.5773502691896258)),
 ('Koninck Studios', ('Image Forum', 0.4082482904638631)),
 ('MHA', ('1992 Number Four Limited Partnership', 1.0)),
 ('Ruby in Paradise', ('Full Crew/Say Yea', 1.0)),
 ('Evzen Kolar Productions',
  ('Alliance Communications Corporation', 0.3333333333333333)),
 ('Village Roadshow Pictures', ('NPV Entertainment', 0.4762120735996621)),
 ('Kanzaman', ('Xuxa Producciones S.L.', 0.30151134457776363)),
 ('Constantin Film Produktion', ('Impa

De esta manera se obtiene la (compania, (similar, distancia coseno))